In [ ]:
# fittingに使う関数
def fitfunc_curve(h, hr, s, a, w, c, d):
    """
    DC電圧の波形の関数．
    
     Parameters
    ----------
    h : float
        印加磁場
    hr : float
        共鳴磁場
    w : float
        線幅
    s : float
        symmetry成分の振幅
    a : float
        antisymmetry成分の振幅
    c : float
        線型部分の傾き
    d : float
        線型部分の切片
    """
    return s * w * w / ((h - hr) * (h - hr) + w * w) + a * w * (h - hr) / ((h - hr)*(h - hr) + w*w) + c*h + d


def fitfunc_kittel(h, Meff):
    """
    面内に磁場をかけた時のKittelの式．
    
     Parameters
    ----------
    h : 印加磁場
    1.76 : 磁気回転比
    Meff : 有効磁化
    """
    return 1.76 * 100 / (2 * np.pi) * np.sqrt(h / 1000 * (h / 1000 + Meff))

def fitfunc_line(x, a, b):
    return a * x + b


In [ ]:
# ファイルの読み込み
import os
import tkinter as tk
import tkinter.filedialog as fd
import re
import glob

def open_folder():
    """
    ダイアログを表示してフォルダを選択
    """
    dir = os.getcwd()
    root = tk.Tk()
    root.withdraw()
    fld = fd.askdirectory(initialdir = dir)
    return fld

foldername = open_folder()
filename = glob.glob(foldername + '/*lvm')  # lvmファイルがあるフォルダー内のファイル名を取得
file_list = []
for i, file in enumerate(filename):
    # ファイル名から周波数部分だけ抽出
    pat = r'[0-9]+\.[0-9]+'  # 抽出するパターン：数字
    num_list = re.findall(pat, file)  # 数字を抽出してリスト化
    fre = float(num_list[0])
    file_list.append([fre, file])

file_list.sort(key=lambda x: x[0])

In [ ]:
Fontsize = 12

# 用いるデータの範囲
use_range = range(10, 1190)

# 負側のフィッティング範囲
n_range = list(range(10, 590))

# 正側のフィッティング範囲
p_range = list(range(610, 1190))

# 電流の周波数
frequency = []
# 共鳴磁場
h_rp = []
h_rn = []

# 線幅
w_rp = []
w_rn = []

# symmetryとantisymmetryの比
saratio_p = []
saratio_n = []

In [ ]:
import scipy.optimize as sopt
import scipy.constants as sconst

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def curve_fitting(fitfunc, x, y, range, para_ini):
    # フィッティング関数の指定
    # フィッティングする範囲
    x_ = x[range]
    y_ = y[range]

    # フィッティングパラメータの最適化
    para_opt, _ = sopt.curve_fit(
        fitfunc, x_, y_, para_ini)

    return para_opt

for i, [fre, filepath] in enumerate(file_list):
    print(filepath)
    df = pd.read_table(filepath, names=['MF', 'Vol'])
    x = df['MF']
    y = df['Vol']
    
    frequency.append(fre)
    # plot raw data
    plt.plot(x[list(use_range)], y[list(use_range)],
             marker="o", label='raw data')

    # plot fitfunc_curve fitting

    # for positive
    para_ini = [17 * fre - 17, 1.0, 1.0, 5.0, 1.0, 1.0]
    para_p = curve_fitting(fitfunc_curve, x, y, p_range, para_ini)
    h = np.arange(min(x[p_range]) - 1,
                      max(x[p_range]) + 1, 0.1)
    f = fitfunc_curve(h, *para_p)
    plt.plot(h, f, color='Red', linewidth=2.0)

    # fitting parameter(positive)
    h_rp.append(para_p[0])
    w_rp.append(para_p[3])
    saratio_p.append(para_p[1] / para_p[2])
    
    # for negative
    para_ini = [-17 * fre + 17, 1.0, 1.0, 5.0, 1.0, 1.0]
    para_n = curve_fitting(fitfunc_curve, x, y, n_range, para_ini)
    h = np.arange(min(x[n_range]) - 1,
                  max(x[n_range]) + 1, 0.1)
    f = fitfunc_curve(h, *para_n)
    plt.plot(h, f, color='Red', linewidth=2.0)

    # fitting parameter(negative)
    h_rn.append(para_n[0])
    w_rn.append(para_n[3])
    saratio_n.append(para_n[1] / para_n[2])

    # プロット設定
plt.xticks(fontsize=Fontsize)
plt.yticks(fontsize=Fontsize)
plt.grid(True)
plt.title("Curve fitting")
plt.xlabel('magnetic field (mT)', fontsize=Fontsize)
plt.ylabel('voltage (V)', fontsize=Fontsize)
plt.savefig(foldername + "/fitting_result.png")
resultForSaving = pd.DataFrame({
})
print(h_rn)
print(frequency)

In [ ]:
#####  kittel negatigve
plt.figure()
plt.plot()

para_ini_kittel = [-0.500]
para_opt_kittel, _ = sopt.curve_fit(
    fitfunc_kittel, h_rn, frequency, para_ini_kittel)  # フィッティングパラメータの最適化

Ms_n = para_opt_kittel[0]
hr_fit = np.arange(-200, 0, 10)
f_fit = fitfunc_kittel(hr_fit, *para_opt_kittel)

plt.plot(h_rn, frequency, marker="o", color="Purple")
plt.plot(hr_fit, f_fit, color='Green', linewidth=2.0,
         label="Meff=" + str(round(Ms_n, 4)))

# プロット設定
plt.xticks(fontsize=Fontsize)
plt.yticks(fontsize=Fontsize)
plt.grid(True)
plt.title("kittel fitting(negative field)")
plt.legend(fontsize=Fontsize + 1, loc="lower center")
plt.xlabel('resonance field (mT)', fontsize=Fontsize)
plt.ylabel('frequency (GHz)', fontsize=Fontsize)
plt.ylim(0, 13)
plt.savefig(foldername + "/kittel_neg_result.png")
#####  kittel positeve
plt.figure()
plt.plot()

para_ini_kittel = [0.500]
para_opt_kittel, _ = sopt.curve_fit(
    fitfunc_kittel, h_rp, frequency, para_ini_kittel)
Ms_p = para_opt_kittel[0]
hr_fit = np.arange(0, 200, 10)
f_fit = fitfunc_kittel(hr_fit, *para_opt_kittel)
plt.plot(h_rp, frequency, marker="o", color="Purple")
plt.plot(hr_fit, f_fit, color='Green', linewidth=2.0,
         label="Meff=" + str(round(Ms_p, 4)))

# プロット設定
plt.xticks(fontsize=Fontsize)
plt.yticks(fontsize=Fontsize)
plt.grid(True)
plt.title("kittel fitting(negative field)")
plt.legend(fontsize=Fontsize + 1, loc="lower center")
plt.xlabel('resonance field (mT)', fontsize=Fontsize)
plt.ylabel('frequency (GHz)', fontsize=Fontsize)
plt.ylim(0, 13)
plt.savefig(foldername + "/kittel_pos_result.png")


In [ ]:
# line width_negative
plt.figure()
plt.plot()

para_ini_line = [2, 0.1]
para_opt_line, cov = sopt.curve_fit(
    fitfunc_line, frequency, w_rn, para_ini_line)  # フィッティングパラメータの最適化

a_n = para_opt_line[0]
inhomo_width_n = para_opt_line[1]

gamma = 1.761 * 10 ** 11

alpha_n = a_n * gamma / 2 / np.pi * 10 ** -12

fre_fit = np.arange(0, 12, 0.5)
w_fit = fitfunc_line(fre_fit, *para_opt_line)

plt.plot(frequency, w_rn, marker="o", color="Darkblue")
plt.plot(fre_fit, w_fit, color='Darkcyan', linewidth=2.0,
         label="alpha=" + str(round(alpha_n, 4)) + " deltaH=" + str(round(inhomo_width_n, 4)))
# プロット設定
plt.xticks(fontsize=Fontsize)
plt.yticks(fontsize=Fontsize)
plt.grid(True)
plt.title("linewidth(negative field)")
plt.legend(fontsize=Fontsize + 1, loc="lower center")
plt.xlabel('frequency (GHz)', fontsize=Fontsize)
plt.ylabel('linewidth', fontsize=Fontsize)
plt.ylim(0, 13)
plt.savefig(foldername + "/line_width_neg_result.png")

plt.ylim(0, 13)

#####  line width_positive
plt.figure()
plt.plot()

para_ini_line = [2, 0.1]
para_opt_line, cov = sopt.curve_fit(
    fitfunc_line, frequency, w_rp, para_ini_line)  # フィッティングパラメータの最適化

a_p = para_opt_line[0]
inhomo_width_p = para_opt_line[1]

alpha_p = a_p * gamma / 2 / np.pi * 10 ** -12

fre_fit = np.arange(0, 12, 0.5)
w_p_fit = fitfunc_line(fre_fit, *para_opt_line)

plt.plot(frequency, w_rp, marker="o", color="Darkblue")
plt.plot(fre_fit, w_p_fit, color='Darkcyan', linewidth=2.0,
         label="alpha=" + str(round(alpha_p, 4)) + " deltaH=" + str(round(inhomo_width_p, 4)))
# プロット設定
plt.xticks(fontsize=Fontsize)
plt.yticks(fontsize=Fontsize)
plt.grid(True)
plt.title("linewidth(negative field)")
plt.legend(fontsize=Fontsize + 1, loc="lower center")
plt.xlabel('frequency (GHz)', fontsize=Fontsize)
plt.ylabel('linewidth', fontsize=Fontsize)
plt.ylim(0, 13)
plt.ylim(0, 13)
plt.savefig(foldername + "/line_width_pos_result.png")

In [ ]:
#####  s/a プロット
plt.figure()
plt.plot()

plt.ylim(-0.5, 0.5)
plt.plot(frequency, saratio_n, marker="o", color="Navy", label="negative")
plt.plot(frequency, saratio_p, marker="o", color="Orangered", label="positive")
plt.legend()

# プロット設定
plt.xticks(fontsize=Fontsize)
plt.yticks(fontsize=Fontsize)
plt.grid(True)
plt.title("ratio of s to a")
plt.xlabel('frequency (GHz)', fontsize=Fontsize)
plt.ylabel('s/a', fontsize=Fontsize)
plt.legend(fontsize=Fontsize + 1, loc="upper right")
plt.ylim(-0.5, 0.5)
plt.savefig(foldername + "/s_to_a_result.png")

In [ ]:
t_FM = 10
d_NM = 10

In [ ]:
#####  ξFMRプロット
xi_FMR_n = []
xi_FMR_p = []

e = 1.602176
hbar = 1.054571
xi_FMR_n = np.array(saratio_n) * e * Ms_n * t_FM * d_NM * \
    hbar * np.sqrt(1 + Ms_n * 1000 / h_rn) / 1000
xi_FMR_n_average = np.mean(xi_FMR_n)

xi_FMR_p = np.array(saratio_p) * e * Ms_p * t_FM * d_NM * \
    hbar * np.sqrt(1 + Ms_p * 1000 / h_rp) / 1000
xi_FMR_p_average = np.mean(xi_FMR_p)

plt.figure()
plt.plot()
plt.plot(frequency, xi_FMR_n, marker="o", color="Navy")
plt.plot(frequency, xi_FMR_p, marker="o", color="Orangered")
plt.axhline(y=xi_FMR_n_average, lw=2, color='Mediumblue',
            label="xi_FMR=" + str(round(xi_FMR_n_average, 4)))
plt.axhline(y=xi_FMR_p_average, lw=2, color='Tomato',
            label="xi_FMR=" + str(round(xi_FMR_p_average, 4)))

plt.xticks(fontsize=Fontsize)
plt.yticks(fontsize=Fontsize)
plt.grid(True)
plt.title("xiFMR")
plt.legend(fontsize=Fontsize + 1, loc='upper center')
plt.xlabel('frequency (GHz)', fontsize=Fontsize)
plt.ylabel('xi_FMR', fontsize=Fontsize)
plt.ylim(-0.5, 0.5)

fitting_result = []
fitting_result.append(
    ["Ms_n", "Ms_p", "xi_FMR_n_average", "xi_FMR_p_average", "alpha_n", "alpha_p", "inhomoH_n", "inhomoH_p"])
fitting_result.append([Ms_n, xi_FMR_n_average, xi_FMR_p_average,
                       alpha_n, alpha_p, inhomo_width_n, inhomo_width_p])
final_result = pd.DataFrame({
    "Ms_n": Ms_n,
    "Ms_p": Ms_p,
    "alpha_n": alpha_n,
    "alpha_p": alpha_p,
    "inhomo_width_n": inhomo_width_n,
    "inhomo_width_p": inhomo_width_p,
    "xi_FMR_n": xi_FMR_n,
    "xi_FMR_p": xi_FMR_p
})
#ResultForSaving_r = ResultForSaving.reset_index()
FinalResultForSaving = pd.concat([final_result], axis=1)
FinalResultForSaving.to_csv(foldername+"/result.csv")